# Input: PROFIL: Reduction de dimmension    
# Modele: TUNING Hyperparmètre d'un modele type LeNet 1D

In [1]:
import json
import random

import warnings
warnings.simplefilter('ignore')

import pandas as pd
import numpy as np

%matplotlib inline
import matplotlib.pyplot as plt

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Model

from contextlib import redirect_stdout
import keras_tuner as kt

from sklearn import preprocessing, model_selection
from sklearn import metrics



2023-12-02 16:10:56.901821: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


# Data preparation

In [2]:
# Acquisition
df_train = pd.read_json('Train_Intensite_H.json')
df_train['Intensite'] = df_train['Intensite'].apply(lambda x: np.array(x)/(np.array(x).max())).tolist()
df_test = pd.read_json('Test_Intensite_H.json')
df_test['Intensite'] = df_test['Intensite'].apply(lambda x: np.array(x)/(np.array(x).max())).tolist()

#Preprocessing
scaler = preprocessing.MinMaxScaler()
x_train_list = [ scaler.fit_transform(np.asarray(i).reshape(-1, 1)) for i in df_train['Intensite']]
y_train_list = [i for i in df_train['Diagnostic']]
x_test_list = [ scaler.fit_transform(np.asarray(i).reshape(-1, 1)) for i in df_test['Intensite']]
y_test_list = [i for i in df_test['Diagnostic']]

#Création des jeu d'entrainement, de validation et de Test
x_train, x_val, y_train, y_val = model_selection.train_test_split(x_train_list, y_train_list, test_size=0.2, random_state=42, shuffle=True)
x_train = np.asarray(x_train).astype(np.float32).reshape(-1, 256, 1)
y_train = np.asarray(y_train).astype(np.float32).reshape(-1, 1)
x_val = np.asarray(x_val).astype(np.float32).reshape(-1, 256, 1)
y_val = np.asarray(y_val).astype(np.float32).reshape(-1, 1)
x_test = np.asarray(x_test_list).astype(np.float32).reshape(-1, 256, 1)
y_test = np.asarray(y_test_list).astype(np.float32).reshape(-1, 1)

# SET UP

### Define simple function to plot all the metrics present in a keras.callbacks.History

In [4]:
  
def plot_history_metrics(history: keras.callbacks.History):
    acc = history.history['accuracy']
    val_acc = history.history['val_accuracy']

    loss = history.history['loss']
    val_loss = history.history['val_loss']

    
    fig = plt.figure(figsize=(12,6))
    ax1 = fig.add_subplot(121)

    ax1.plot(acc, label='Training Accuracy')
    ax1.plot(val_acc, label='Validation Accuracy')
    ax1.legend(loc='lower right')
    ax1.set_title('Training and Validation Accuracy')

    ax2 = fig.add_subplot(122)
    ax2.plot(loss, label='Training Loss')
    ax2.plot(val_loss, label='Validation Loss')
    ax2.legend(loc='upper right')
    ax2.set_title('Training and Validation Loss')
    plt.show()
    
    fig.savefig("Training and Validation.png")
    plt.close(fig)
    

### build_model function to generate 1D LeNet Like

In [5]:
def build_model(hp):

    # Model Variables
    loss = keras.losses.BinaryCrossentropy()
    thresh = hp.Float("thresh", min_value=0.3, max_value=0.45, step= 0.05)
    l_rate = hp.Float("lr", min_value=1e-4, max_value=1e-2, sampling="log")
    adam = tf.keras.optimizers.Adam(learning_rate= l_rate)
    metric = tf.keras.metrics.BinaryAccuracy(name ='accuracy', threshold= thresh)


    # Model architecture    
    input_layer = keras.Input(shape=(256, 1))

    x = layers.Conv1D(filters=256, kernel_size=3, strides=2, activation=hp.Choice(f"activation_c0", ["relu", "tanh"]), padding="same")(input_layer)

    for i in range (hp.Int("num_conv_layer", 0, 5, step=1)):
        x = layers.Conv1D( filters=hp.Int("filter_conv_0", 32, 1024, sampling='log', step=2), 
                          kernel_size=5, strides=2, activation=hp.Choice(f"activation_c{i+1}", ["relu", "tanh"]), padding="same")(x)
    
    x = layers.BatchNormalization()(x)
    x = layers.Flatten()(x)

    x = layers.Dense(256, activation=hp.Choice(f"activation_d0", ["relu", "tanh"]), kernel_regularizer=keras.regularizers.L2())(x)
        
    for i in range (hp.Int("num_hidden_layer", 1, 5, step=1)):
        if hp.Boolean(f"dropout_{i}"):
            x = layers.Dropout(0.2)(x)

        x = layers.Dense(units=hp.Int(f"units_{i+1}", min_value=32, max_value=512, step=32),
                         activation=hp.Choice(f"activation_d{i+1}", ["relu", "tanh"]),
                         kernel_regularizer=keras.regularizers.L2())(x)

    output_layer = layers.Dense(1, activation="relu")(x)

    model = Model(inputs=input_layer, outputs=output_layer)

    # Modele compilation
    model.compile(optimizer=adam, loss=loss, metrics=metric)
    
    
    return model 



### Instanciate main variables: callbacks, optimizer, loss , metrics, epoch

In [7]:
PROJECT_NAME = "1D-LeNet-like Tuning"


PATH_DIR = "./Tuning"


callbacks = [
    tf.keras.callbacks.ModelCheckpoint("best_model.h5", save_best_only=True, monitor="val_accuracy"),
    tf.keras.callbacks.ReduceLROnPlateau( monitor="val_accuracy", factor=0.2, patience=10, min_lr=0.000001),
    tf.keras.callbacks.EarlyStopping( monitor="val_accuracy", patience=8, verbose=1, mode="auto", restore_best_weights=True, start_from_epoch=10)]


hp = kt.HyperParameters()
model = build_model(hp)

    

Fine Tuning des hyper paramétres

In [8]:
tuner = kt.Hyperband(
    hypermodel=build_model,
    objective="val_accuracy",
    max_epochs=15,
    factor=2,
    directory=PATH_DIR,
    overwrite=True,
    project_name=PROJECT_NAME)



tuner.search(x_train, y_train, epochs=50, validation_data=(x_val, y_val), callbacks=callbacks)


Trial 46 Complete [00h 04m 14s]
val_accuracy: 0.6811145544052124

Best val_accuracy So Far: 0.8250774145126343
Total elapsed time: 02h 59m 42s


### sauvegarde de la meilleures architecture

In [ ]:
help(tunerSearch

In [10]:


with open(PATH_DIR + '/tuning.txt', 'w') as f:
    with redirect_stdout(f):
        print('BEST MODEL-------------')
        print('-----------------------')
        tuner.results_summary(1)
        print('SPACE SEARCH-------------')
        print('-----------------------')
        tuner.search_space_summary()
    f.close()


In [12]:
best_hps=tuner.get_best_hyperparameters()[0]

# Build the model with the optimal hyperparameters and train it on the data for 50 epochs
model = tuner.hypermodel.build(best_hps)
history = model.fit(x_train, y_train, epochs=250, validation_data=(x_val, y_val))

val_acc_per_epoch = history.history['val_accuracy']
best_epoch = val_acc_per_epoch.index(max(val_acc_per_epoch)) + 1
print('Best epoch: %d' % (best_epoch,))

Epoch 1/250
162/162 [==============================] - 14s 74ms/step - loss: 10.0181 - accuracy: 0.6431 - val_loss: 9.3238 - val_accuracy: 0.5015
Epoch 2/250
162/162 [==============================] - 12s 72ms/step - loss: 7.6486 - accuracy: 0.6780 - val_loss: 7.3172 - val_accuracy: 0.5015
Epoch 3/250
162/162 [==============================] - 12s 74ms/step - loss: 6.2202 - accuracy: 0.6483 - val_loss: 5.7798 - val_accuracy: 0.7337
Epoch 4/250
162/162 [==============================] - 12s 72ms/step - loss: 5.1690 - accuracy: 0.7361 - val_loss: 4.7973 - val_accuracy: 0.7376
Epoch 5/250
162/162 [==============================] - 12s 73ms/step - loss: 4.4511 - accuracy: 0.7459 - val_loss: 4.1085 - val_accuracy: 0.8127
Epoch 6/250
162/162 [==============================] - 12s 72ms/step - loss: 4.0041 - accuracy: 0.6867 - val_loss: 3.6857 - val_accuracy: 0.7005
Epoch 7/250
162/162 [==============================] - 12s 72ms/step - loss: 3.5790 - accuracy: 0.7483 - val_loss: 3.7245 - val_a

In [15]:
# Importer MLflow
import mlflow
import mlflow.keras


try:
    experiment_id = mlflow.create_experiment('TUNING input Profil  ouput: Malade/Normal')
except:
    experiment_name = 'TUNING input Profil  ouput: Malade/Normal'
    current_experiment=dict(mlflow.get_experiment_by_name(experiment_name))
    experiment_id=current_experiment['experiment_id']


with mlflow.start_run(experiment_id =experiment_id) as run:

    

    hypermodel = tuner.hypermodel.build(best_hps)
    keras.utils.plot_model(hypermodel,to_file="Model.png", show_shapes=True)
    
    mlflow.log_artifact(PATH_DIR + '/Tuning.txt')
    mlflow.log_artifact("Model.png")

    mlflow.log_param("Seuil", best_hps.get('thresh'))
    mlflow.log_param("loss", 'BinaryCrossentropy')
    mlflow.log_param("learning rate", best_hps.get('lr'))
    mlflow.log_param("epoch", best_epoch)
    mlflow.log_param("CNN", best_hps.get('num_conv_layer')+1)
    mlflow.log_param("DNN", best_hps.get('num_hidden_layer'))
    
    history = hypermodel.fit(x_train, y_train, epochs=best_epoch, validation_data=(x_val, y_val))
    plot_history_metrics(history)

    mlflow.log_artifact("Training and Validation.png")

    eval_result = hypermodel.evaluate(x_test, y_test)
    y_pred = hypermodel.predict(x_test)
    y_pred_class = np.apply_along_axis(lambda x: 1 if x>best_hps.get('thresh') else 0, 1, y_pred)

    confusion_m = metrics.confusion_matrix(y_test, y_pred_class)
                
    VN = confusion_m[0,0]
    VP = confusion_m[1,1]
    FN = confusion_m[1,0]
    FP = confusion_m[0,1]

    plot_history_metrics(history)

    
    print('Justesse du jeu de test =', np.round((VP+VN)/(VP+VN+FP+FN)*100,2),'%')
    print('Sensibilité =', np.round(VP*100/(VP+FN),2),'%')
    print('Spécificité =', np.round(VN*100/(VN+FP),2),'%')

    print('---------Matrice de confusition-----------------')
    print('               pred:Malade     pred:Normal')
    print('vrai Malade:      ',VP,'            ',FN)
    print('vrai Normal       ',FP,'            ',VN)
        
    signature = mlflow.models.infer_signature(x_train, hypermodel.predict(x_train))
    mlflow.keras.log_model(hypermodel, artifact_path="keras-model", signature=signature)

#    mlflow.log_artifact("Training and Validation.png")
    mlflow.log_metric("Justesse",(VP+VN)/(VP+VN+FP+FN))
    mlflow.log_metric("VN", VN)
    mlflow.log_metric("VP", VP)
    mlflow.log_metric("FN", FN)
    mlflow.log_metric("FP", FP)
    mlflow.log_metric("Sensibilite", VP/(VP+FN))
    mlflow.log_metric("Specificite", VN/(VN+FP))
    mlflow.log_param("seuil", seuil)

    mlflow.end_run()

    model_uri = f"runs:/{run.info.run_id}/keras-model"
    mv = mlflow.register_model(model_uri, "Tuning 1D Le Net-like binar")
        
        

You must install pydot (`pip install pydot`) and install graphviz (see instructions at https://graphviz.gitlab.io/download/) for plot_model to work.


OSError: [Errno 30] Read-only file system: '/stevecostalat'